In [1]:
import requests
import xml.etree.ElementTree as ET
from collections import defaultdict
import time

In [2]:
def get_token():
    #get access token
    url = 'https://webservices.bvdep.com/orbis/remoteaccess.asmx'
    headers = {'Content-Type': 'text/xml'}
    data = """<?xml version="1.0" encoding="utf-8"?>
    <soap12:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap12="http://www.w3.org/2003/05/soap-envelope">
      <soap12:Body>
        <Open xmlns="http://bvdep.com/webservices/">
          <username>iGenieWS123</username>
          <password>jnecyu4wa!</password>
        </Open>
      </soap12:Body>
    </soap12:Envelope>"""

    response = requests.post(url, headers=headers, data=data)

    if response.status_code != requests.codes.ok:
        print(response.text)
        return None

    OpenResponse = ET.fromstring(response.text)
    token = OpenResponse[0][0][0].text
    print(token)
    return token


In [3]:
def close_connection(token):
    #get access token
    url = 'https://webservices.bvdep.com/orbis/remoteaccess.asmx'
    headers = {'Content-Type': 'text/xml'}
    data = """<?xml version="1.0" encoding="utf-8"?>
<soap12:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap12="http://www.w3.org/2003/05/soap-envelope">
  <soap12:Body>
    <Close xmlns="http://bvdep.com/webservices/">
      <sessionHandle>{}</sessionHandle>
    </Close>
  </soap12:Body>
</soap12:Envelope>""".format(token)

    response = requests.post(url, headers=headers, data=data)

In [4]:
def find_by_bvd_id(token):
    url = 'https://webservices.bvdep.com/orbis/remoteaccess.asmx'
    headers = {'Content-Type': 'text/xml'}
    #FindByBvdId
    data = """<?xml version="1.0" encoding="utf-8"?>
    <soap12:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap12="http://www.w3.org/2003/05/soap-envelope">
      <soap12:Body>
        <FindByBVDId xmlns="http://bvdep.com/webservices/">
          <sessionHandle>{}</sessionHandle>
          <id>{}</id>
        </FindByBVDId>
      </soap12:Body>
    </soap12:Envelope>""".format(token,"DE8170003036")

    response = requests.post(url, headers=headers, data=data)

    if response.status_code != requests.codes.ok:
        print(response.text)
        return None

    FindByBvdId = ET.fromstring(response.text)
    SelectionResult_token = FindByBvdId[0][0][0][0].text
    print(SelectionResult_token)
    return SelectionResult_token

In [5]:
def get_data(token, SelectionResult_token, query, field, constituent):        
    url = 'https://webservices.bvdep.com/orbis/remoteaccess.asmx'
    headers = {'Content-Type': 'text/xml'}
    data = """<?xml version="1.0" encoding="utf-8"?>
<soap12:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap12="http://www.w3.org/2003/05/soap-envelope">
  <soap12:Body>
    <GetData xmlns="http://bvdep.com/webservices/">
      <sessionHandle>{0}</sessionHandle>
      <selection>
        <Token>{1}</Token>
        <SelectionCount>1</SelectionCount>
      </selection>
      <query>{2}</query>
      <fromRecord>0</fromRecord>
      <nrRecords>5</nrRecords>
      <resultFormat>CSV</resultFormat>
    </GetData>
  </soap12:Body>
</soap12:Envelope>""".format(token,SelectionResult_token,query)

    response = requests.post(url, headers=headers, data=data)

    if response.status_code != requests.codes.ok:
        print(response.text)
        with open("./results/failed.txt", 'a') as f:
            f.write("{} \n".format(field))
        return None
    
    with open("./results/success.txt", 'a') as f:
            f.write("{} \n".format(field))

    result = response.text
    file_name = "./results/{}_{}.xml".format(constituent,field)
    with open(file_name, 'w') as f:
        f.write(result)
    return True

In [6]:
def get_available_fields():
    all_fields = defaultdict(list)

    available_fields = ET.parse(r'available_fields.xml')
    fields = available_fields.getroot()

    #i = 0
    for child in fields:
        #if i == 20:
            #break
        if len(child) > 0:
            document = {"code":child.attrib["code"], "dimension":child[0].attrib["default"]}
        else:
            document = {"code":child.attrib["code"]}

        all_fields[child.attrib["code"].split("_")[0]].append(document)
        #i += 1  
    return all_fields

In [7]:
def main():
    errors = 0
    all_fields = get_available_fields()
    token = get_token()
    print(token)
    selection_token = find_by_bvd_id(token)
    
    for key in all_fields.keys():
        if errors == 2:
            print("2 errors")
            return
        
        query = "SELECT "
        middle = []
        for item in all_fields[key]:
            part = item["code"] + " "
            if "dimension" in item:
                part += "USING [Parameters.RepeatingDimension={};]".format(item["dimension"])
            middle.append(part)
        query += ", ".join(middle) + " FROM RemoteAccess.A"

        get_data_result = get_data(token,selection_token,query,key,"BMW")
        
        if not get_data_result:
            errors += 1
            close_connection(token)
            time.sleep(900)
            token = get_token()
            if not token:
                print("Couldn't get second token")
                return
            print(token)
            selection_token = find_by_bvd_id(token)
            

        time.sleep(60)

In [8]:
main()

3L8JEE24B37CB9B
3L8JEE24B37CB9B
0
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><soap:Fault><soap:Code><soap:Value>soap:Receiver</soap:Value></soap:Code><soap:Reason><soap:Text xml:lang="en">System.Web.Services.Protocols.SoapException: Server was unable to process request. ---&gt; Credits limitations
   at System.Web.Services.Protocols.SoapHttpClientProtocol.ReadResponse(SoapClientMessage message, WebResponse response, Stream responseStream, Boolean asyncCall)
   at System.Web.Services.Protocols.SoapHttpClientProtocol.Invoke(String methodName, Object[] parameters)
   at ProxyRemoteAccess.com.bvdep.webservices.RemoteAccess.GetData(String sessionHandle, SelectionResult selection, String query, Int32 fromRecord, Int32 nrRecords, ResultFormatting resultFormat) in Q:\QuickFixes\Releases\Net\webservices.bvdep.com\RootVersion